In [ ]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

# Making a Pandas Dataframe from the CSV Files

In [ ]:
train_data = pd.read_csv('train.csv')
print(train_data.shape)
train_data.head()

# Splitting the Data and Labels

In [ ]:
splitting_factor = 0.8

x_train = train_data.iloc[ : int( len(x_train) * splitting_factor), 1 :]
x_label = train_data.iloc[:int( len(x_train) * splitting_factor), :1]

y_train = train_data.iloc[int( len(x_train) * splitting_factor):, 1:]
y_label = train_data.iloc[int( len(x_train) * splitting_factor):, :1]

# Converting Data into numpy arrays to feed it into the network 

In [ ]:
x_label_arr = x_label.to_numpy()
x_label_arr = x_label_arr.flatten()

y_label_arr = y_label.to_numpy()
y_label_arr = y_label_arr.flatten()

x_train = x_train.to_numpy()
x_train = x_train.reshape(-1,28,28,1)
x_train = np.pad(x_train, ((0,0), (2,2), (2, 2),(0,0)), 'constant')

y_train = y_train.to_numpy()
y_train = y_train.reshape(-1,28,28,1)
y_train = np.pad(y_train, ((0,0), (2,2), (2, 2),(0,0)), 'constant')

# One Hot Encoding the training Labels  

In [ ]:
x_label_hot = np.zeros((x_label_arr.size, x_label_arr.max()+1))
x_label_hot[np.arange(x_label_arr.size),x_label_arr] = 1

y_label_hot = np.zeros((y_label_arr.size, y_label_arr.max()+1))
y_label_hot[np.arange(y_label_arr.size),y_label_arr] = 1

x_label_hot = x_label_hot.reshape(-1,1,1,10)
y_label_hot = y_label_hot.reshape(-1,1,1,10)

# Building the model

Lenet 5

In [ ]:
model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(filters = 6, kernel_size = (5,5), activation='relu', input_shape=(32, 32, 1)),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
            tf.keras.layers.Conv2D(filters = 16, kernel_size = (5,5), activation='relu', input_shape=(14, 14, 6)),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
            tf.keras.layers.Conv2D(filters = 120, kernel_size = (5,5), activation='relu', input_shape=(5 ,5 ,16)),
            tf.keras.layers.Dense(84, activation=tf.nn.relu),
            tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

model.compile(optimizer = tf.keras.optimizers.Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

In [ ]:
annealer = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

In [ ]:
history = model.fit_generator(datagen.flow(x_train, x_label_hot, batch_size=16),
                           steps_per_epoch=500,
                           epochs=100, 
                           verbose=2,  
                           validation_data=(y_train,y_label_hot), 
                           callbacks=[annealer])

# Evaluating our Model

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Predicting some previously Unseen Digits